# Llama -2 RAG on IX History text

## Package Import

In [1]:
import os
import torch
import pinecone
from tqdm import tqdm

from langchain.vectorstores import Pinecone
from langchain.llms import HuggingFacePipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader




/home/anshuman/workspace/Aaraki/venv/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
!nvidia-smi

Sun Oct  1 00:04:36 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:17:00.0 Off |                  N/A |
|  0%   49C    P8               31W / 350W|     20MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Prepare Llama-2 model

In [4]:
from torch import bfloat16
import transformers

model_id = 'meta-llama/Llama-2-13b-chat-hf'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

hf_auth = os.environ.get('HUGGINGFACE_ENV') or 'HUGGINGFACE_ENV'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth,
    resume_download = True

)
model.eval()
print(f"Model loaded on {device}")

/home/anshuman/workspace/Aaraki/venv/lib/python3.8/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.96s/it]


Model loaded on cuda


In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/home/anshuman/workspace/Aaraki/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


## Prepare Embedding Object

In [6]:
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

embed_model = HuggingFaceEmbeddings(
    model_name = embed_model_id,
    model_kwargs = {'device': device},
    # encode_kwargs = {'device':device, 'batch_size':32}
)

In [7]:
# test embedding
docs = [
    "Kill him a and all that it takes",
    "why is he like that?"
]

embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 2 doc embeddings, each with a dimensionality of 384.


## Pinecone (Vector DB)

In [8]:


# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY',
    environment = os.environ.get('PINECONE_ENV') or 'PINECONE_ENV',
)

#### Initiate Pinecone Index

In [9]:
import time

index_name = 'ix-history-2'

if index_name not in pinecone.list_indexes():
  pinecone.create_index(
      index_name,
      dimension=len(embeddings[0]),
      metric='cosine'
  )

  while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

  print("Index Ready")

In [10]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00402,
 'namespaces': {'': {'vector_count': 402}},
 'total_vector_count': 402}

#### Loading Raw data

In [11]:
pdfLocation = "../datasets/Social/iess302.pdf"

loader = PyPDFLoader(pdfLocation)
data = loader.load()

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
texts = text_splitter.split_documents(data)

print (f'Now you have {len(texts)} documents')

Now you have 66 documents


### Move embedding to Pinecone Index

In [13]:

ids = [f"{i}-{texts[i].metadata['page']}" for i in range(len(texts))]
textList = [text.page_content for text in texts]
embeds = embed_model.embed_documents(textList) 
metadata = [{
    'text': text.page_content,
         'source':text.metadata['source'],
         'page': text.metadata['page']
} for text in texts]

index.upsert(vectors=zip(ids, embeds, metadata), show_progress=True)

print('Upsert Done!')

Upsert Done!


## Preparing LLM Pipeline

In [14]:

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    framework="pt",
)

In [15]:
llm = HuggingFacePipeline(pipeline=generate_text)

#### hf RAG Pipeline

In [16]:
text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

In [17]:
query = "What were the social, economic and political conditions in Russia before 1905?"

vectorstore.similarity_search(query, k=6)
print(vectorstore)

In [18]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [19]:
content = rag_pipeline('What were the social, economic and political conditions in Russia before 1905?')
print(content['result'])

 Before 1905, Russia had a feudal system with a small landowning class and a large peasantry. The country was also experiencing rapid industrialization and urbanization, leading to growing social and economic disparities between the wealthy elite and the poor masses. Additionally, the government was highly autocratic and repressive, with limited political freedoms for the population. These factors contributed to the growing unrest and discontent among the people, which eventually led to the Russian Revolution in 1905.
